In [3]:
import tvm
import pandas as pd
%pylab
df=pd.read_pickle('/home/jaehun/workspace/pytorch-maml/data/result5.pkl',compression='gzip')


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [14]:
df

,names,knobs,weights,strides,paddings,indexs,targets,networks,vectors,cost
0,topi_x86_conv2d_NCHWc,tile_ic-tile_oc-tile_ow-unroll_kw,"(64, 3, 3, 3)","(1, 1)","(1, 1, 1, 1)",220,llvm -mcpu=core-avx2,vgg-13,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 3.0, 0.0, ...",0.0532587
1,topi_x86_conv2d_NCHWc,tile_ic-tile_oc-tile_ow-unroll_kw,"(64, 3, 3, 3)","(1, 1)","(1, 1, 1, 1)",92,llvm -mcpu=core-avx2,vgg-13,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 3.0, 0.0, ...",0.0496674
2,topi_x86_conv2d_NCHWc,tile_ic-tile_oc-tile_ow-unroll_kw,"(64, 3, 3, 3)","(1, 1)","(1, 1, 1, 1)",229,llvm -mcpu=core-avx2,vgg-13,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0.0339944
3,topi_x86_conv2d_NCHWc,tile_ic-tile_oc-tile_ow-unroll_kw,"(64, 3, 3, 3)","(1, 1)","(1, 1, 1, 1)",200,llvm -mcpu=core-avx2,vgg-13,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0.0329682
4,topi_x86_conv2d_NCHWc,tile_ic-tile_oc-tile_ow-unroll_kw,"(64, 3, 3, 3)","(1, 1)","(1, 1, 1, 1)",113,llvm -mcpu=core-avx2,vgg-13,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 3.0, 0.0, ...",0.0328156
...,...,...,...,...,...,...,...,...,...,...
535,topi_nn_conv2d,tile_f-tile_y-tile_x-tile_rc-tile_ry-tile_rx-a...,"(512, 512, 3, 3)","(1, 1)","(1, 1, 1, 1)",814404,cuda -model=unknown,vgg-13,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 3.0, 26.78...",Traceback (most recent call last):\n [bt] (3)...
536,topi_nn_conv2d,tile_f-tile_y-tile_x-tile_rc-tile_ry-tile_rx-a...,"(512, 512, 3, 3)","(1, 1)","(1, 1, 1, 1)",820524,cuda -model=unknown,vgg-13,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 26.78...",Traceback (most recent call last):\n [bt] (3)...
537,topi_nn_conv2d,tile_f-tile_y-tile_x-tile_rc-tile_ry-tile_rx-a...,"(512, 512, 3, 3)","(1, 1)","(1, 1, 1, 1)",3537747,cuda -model=unknown,vgg-13,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.19264507...",Traceback (most recent call last):\n [bt] (3)...
538,topi_nn_conv2d,tile_f-tile_y-tile_x-tile_rc-tile_ry-tile_rx-a...,"(512, 512, 3, 3)","(1, 1)","(1, 1, 1, 1)",7901637,cuda -model=unknown,vgg-13,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.19264507...",Traceback (most recent call last):\n [bt] (3)...


In [259]:
_dataset=[]
num=0
for idx,value in df.groupby(['shots']):
#     print(idx)
    value['idx']=num
    value['len']=value['vectors'].map(lambda x: len(x))
    _dataset.append(value[['idx','vectors','cost','len']].values)
    num+=1
    
dataset=np.concatenate(_dataset)


ValueError: need at least one array to concatenate

In [262]:
train_shot=5
test_shot=5
way=5

In [250]:
if np.random.randint(2)==1:
    test=np.array(np.vstack(dataset[dataset[:,3]==481][:,0]))
    data=np.array(np.vstack(dataset[dataset[:,3]==481][:,1]))
    labels=np.array(np.vstack(dataset[dataset[:,3]==481][:,2]))
    ways=np.random.default_rng().choice(list(set(test.reshape(-1))), size=way, replace=False)
else:
    test=np.array(np.vstack(dataset[dataset[:,3]!=481][:,0].tolist()))
    data=np.array(np.vstack(dataset[dataset[:,3]!=481][:,1].tolist()))
    labels=np.array(np.vstack(dataset[dataset[:,3]!=481][:,2].tolist()))
    ways=np.random.default_rng().choice(list(set(test.reshape(-1))), size=way, replace=False)

train_data=[]
train_labels=[]
test_data=[]
test_labels=[]
for i in ways:
    idx=np.random.choice(np.where(test==i)[0],train_shot)
    train_data.append(data[idx])
    train_labels.append(labels[idx])

    idx=np.random.choice(np.where(test==i)[0],test_shot)
    test_data.append(data[idx])
    test_labels.append(labels[idx])

train_data=np.vstack(train_data)
train_labels=np.vstack(train_labels)
test_data=np.vstack(test_data)
test_labels=np.vstack(test_labels)
